In [8]:
from kafka import KafkaProducer
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import pandas as pd
import json

import tkinter as tk

import requests
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

In [9]:
#Retourne le code source de l'URL qui est passé en paramètre

def get_source(url):
    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [10]:
#Retourne un dataFrame qui contient le contenu du flux RSS passé en paramètre

def get_feed(url):   
    response = get_source(url)
    
    df = pd.DataFrame(columns = ['title', 'pubDate', 'link', 'description'])

    with response as r:
        items = r.html.find("item", first=False)

        for item in items:        

            title = item.find('title', first=True).text
            pubDate = item.find('pubDate', first=True).text
            link = item.find('guid', first=True).text
            description = item.find('description', first=True).text


            row = {'title': title, 'pubDate': pubDate, 'link': link, 'description': description}
            df = df.append(row, ignore_index=True)

    return df

In [11]:
#Permet à l'utilisateur de sélectionner les sites internet qu'il veut consulter parmi une liste prédéfinie
#Retourne une liste qui contient ces sites 
def ouvrir_fenetre():
    sites = {
        "Le Monde": "https://www.lemonde.fr/rss/une.xml",
        "OVH Cloud": "https://blog.ovhcloud.com/feed/",
        "Actu IA": "https://www.actuia.com/feed/",
        "20 Minutes": "https://www.20minutes.fr/feeds/rss-une.xml",
        "France 24":"https://www.france24.com/fr/rss"
    }

    def ajouter_site():
        selection.clear()
        for site in listebox.curselection():
            nom_site = listebox.get(site)
            selection.append(sites[nom_site])
        fenetre.quit()

    fenetre = tk.Tk()

    listebox = tk.Listbox(fenetre, selectmode=tk.MULTIPLE)
    for site in sites:
        listebox.insert(tk.END, site)

    bouton_ajouter = tk.Button(fenetre, text="Ajouter", command=ajouter_site)

    selection = []

    listebox.grid(row=0, column=0)
    bouton_ajouter.grid(row=1, column=0)

    fenetre.mainloop()

    return selection

In [12]:
#Appel de la fonction
urls=ouvrir_fenetre()
urls

['https://www.lemonde.fr/rss/une.xml', 'https://blog.ovhcloud.com/feed/']

In [13]:
# Création du dataFrame en fonction des sites sélectionnés
dfs = []
for url in urls:
    df = get_feed(url)
    dfs.append(df)

# Fusion des dataFrame en un seul
data = pd.concat(dfs, ignore_index=True)


/Users/antoinesusini/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
/var/folders/kb/kq8rbcl93hq1t3bmnp4b0yj00000gn/T/ipykernel_50825/1732422673.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/var/folders/kb/kq8rbcl93hq1t3bmnp4b0yj00000gn/T/ipykernel_50825/1732422673.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat ins

,title,pubDate,link,description
0,<![CDATA[Manifestations du 6 avril contre la r...,"Thu, 06 Apr 2023 14:53:44 +0200",https://www.lemonde.fr/politique/live/2023/04/...,<![CDATA[Des manifestations ont à nouveau eu l...
1,<![CDATA[Guerre en Ukraine en direct : la Russ...,"Thu, 06 Apr 2023 11:25:06 +0200",https://www.lemonde.fr/international/live/2023...,"<![CDATA[Emmanuel Macron, en visite en Chine, ..."
2,<![CDATA[Le rapport accablant sur les choix én...,"Thu, 06 Apr 2023 11:00:03 +0200",https://www.lemonde.fr/economie/article/2023/0...,"<![CDATA[Après six mois de travail, la commiss..."
3,"<![CDATA[Le lobby agroalimentaire breton, une ...","Thu, 06 Apr 2023 05:00:12 +0200",https://www.lemonde.fr/societe/article/2023/04...,"<![CDATA[« En Bretagne, main basse sur la terr..."
4,<![CDATA[Guerre en Ukraine : les forces russes...,"Thu, 06 Apr 2023 12:26:54 +0200",https://www.lemonde.fr/international/article/2...,"<![CDATA[Depuis plusieurs mois, les Russes cre..."
5,<![CDATA[Qui contrôle la police dans les autre...,"Wed, 05 Apr 2023 15:35:21 +0200",https://www.lemonde.fr/les-decodeurs/article/2...,"<![CDATA[Ombudsman suédois, médiateur régional..."
6,"<![CDATA[Le vol de vélos, un obstacle à la pra...","Thu, 06 Apr 2023 11:31:44 +0200",https://www.lemonde.fr/economie/article/2023/0...,<![CDATA[Seulement 61 % des victimes rachètent...
7,<![CDATA[Quelles quantités d’eau sont prélevée...,"Sat, 01 Apr 2023 06:00:21 +0200",https://www.lemonde.fr/les-decodeurs/article/2...,<![CDATA[Près de la moitié des 32 milliards de...
8,<![CDATA[Grand Paris Express : une filiale de ...,"Thu, 06 Apr 2023 09:39:53 +0200",https://www.lemonde.fr/societe/article/2023/04...,"<![CDATA[Maxime Wagner, un intérimaire de 37 a..."
9,"<![CDATA[Caroline Garcia, en perte de vitesse,...","Thu, 06 Apr 2023 13:00:34 +0200",https://www.lemonde.fr/sport/article/2023/04/0...,<![CDATA[Après une séparation soudaine fin oct...


In [22]:
#Nettoyage des données 

data['title'] = data['title'].apply(lambda x: x.replace('ê', 'e'))
data['title'] = data['title'].apply(lambda x: x.replace("'", ""))
data['description'] = data['description'].apply(lambda x: x.replace("'", ""))
for i in range(len(data)):
    data['description'] = data['description'].apply(lambda x: x.replace('ê', 'e'))
    if '[' in data.loc[i,'title'] and ']' in data.loc[i,'title']:
        data.loc[i,'title'] = data.loc[i,'title'].split('<![CDATA[')[1].split(']')[0] 
    if '[' in data.loc[i,'description'] and ']' in data.loc[i,'description']:
        data.loc[i,'description'] = data.loc[i,'description'].split('<![CDATA[')[1].split(']')[0] 
    else:
        data.loc[i,'title'] = data.loc[i,'title']
        data.loc[i,'description'] = data.loc[i,'description']


,title,pubDate,link,description
0,Manifestations du 6 avril contre la réforme de...,2023-04-06 14:53:44+02:00,https://www.lemonde.fr/politique/live/2023/04/...,Des manifestations ont à nouveau eu lieu dans ...
1,"En Côte d’Ivoire, le parti du président Alassa...",2023-04-06 14:30:41+02:00,https://www.lemonde.fr/afrique/article/2023/04...,"Alors que se profilent les élections locales, ..."
2,"En manque de pluies, Mayotte va de plus en plu...",2023-04-06 14:20:14+02:00,https://www.lemonde.fr/planete/article/2023/04...,Avec un déficit pluviométrique exceptionnel et...
3,"Caroline Garcia, en perte de vitesse, rappelle...",2023-04-06 13:00:34+02:00,https://www.lemonde.fr/sport/article/2023/04/0...,Après une séparation soudaine fin octobre 2022...
4,Guerre en Ukraine : les forces russes fortifie...,2023-04-06 12:26:54+02:00,https://www.lemonde.fr/international/article/2...,"Depuis plusieurs mois, les Russes creusent des..."
5,"Le vol de vélos, un obstacle à la pratique",2023-04-06 11:31:44+02:00,https://www.lemonde.fr/economie/article/2023/0...,Seulement 61 % des victimes rachètent une bicy...
6,"Après la mort brutale de Jean-Paul Capitani, e...",2023-04-06 11:28:21+02:00,https://www.lemonde.fr/disparitions/article/20...,"Depuis plusieurs décennies, l’éditeur, entrepr..."
7,Guerre en Ukraine en direct : la Russie exclut...,2023-04-06 11:25:06+02:00,https://www.lemonde.fr/international/live/2023...,"Emmanuel Macron, en visite en Chine, a appelé,..."
8,Le rapport accablant sur les choix énergétique...,2023-04-06 11:00:03+02:00,https://www.lemonde.fr/economie/article/2023/0...,"Après six mois de travail, la commission d’enq..."
9,L’Iran et l’Arabie saoudite normalisent leurs ...,2023-04-06 09:52:01+02:00,https://www.lemonde.fr/international/article/2...,Téhéran et Riyad avaient rompu leurs liens déb...


In [23]:
#Classement des articles par ordre décroissant (plus récents en premier)
data['pubDate'] = pd.to_datetime(data['pubDate'], format='%a, %d %b %Y %H:%M:%S %z')

data = data.sort_values('pubDate', ascending=False)

data = data.reset_index(drop=True)

ValueError: time data '2023-04-06 14:53:44+02:00' does not match format '%a, %d %b %Y %H:%M:%S %z' (match)

In [24]:
data

,title,pubDate,link,description
0,Manifestations du 6 avril contre la réforme de...,2023-04-06 14:53:44+02:00,https://www.lemonde.fr/politique/live/2023/04/...,Des manifestations ont à nouveau eu lieu dans ...
1,"En Côte d’Ivoire, le parti du président Alassa...",2023-04-06 14:30:41+02:00,https://www.lemonde.fr/afrique/article/2023/04...,"Alors que se profilent les élections locales, ..."
2,"En manque de pluies, Mayotte va de plus en plu...",2023-04-06 14:20:14+02:00,https://www.lemonde.fr/planete/article/2023/04...,Avec un déficit pluviométrique exceptionnel et...
3,"Caroline Garcia, en perte de vitesse, rappelle...",2023-04-06 13:00:34+02:00,https://www.lemonde.fr/sport/article/2023/04/0...,Après une séparation soudaine fin octobre 2022...
4,Guerre en Ukraine : les forces russes fortifie...,2023-04-06 12:26:54+02:00,https://www.lemonde.fr/international/article/2...,"Depuis plusieurs mois, les Russes creusent des..."
5,"Le vol de vélos, un obstacle à la pratique",2023-04-06 11:31:44+02:00,https://www.lemonde.fr/economie/article/2023/0...,Seulement 61 % des victimes rachètent une bicy...
6,"Après la mort brutale de Jean-Paul Capitani, e...",2023-04-06 11:28:21+02:00,https://www.lemonde.fr/disparitions/article/20...,"Depuis plusieurs décennies, l’éditeur, entrepr..."
7,Guerre en Ukraine en direct : la Russie exclut...,2023-04-06 11:25:06+02:00,https://www.lemonde.fr/international/live/2023...,"Emmanuel Macron, en visite en Chine, a appelé,..."
8,Le rapport accablant sur les choix énergétique...,2023-04-06 11:00:03+02:00,https://www.lemonde.fr/economie/article/2023/0...,"Après six mois de travail, la commission d’enq..."
9,L’Iran et l’Arabie saoudite normalisent leurs ...,2023-04-06 09:52:01+02:00,https://www.lemonde.fr/international/article/2...,Téhéran et Riyad avaient rompu leurs liens déb...


In [38]:
# Configure Kafka
bootstrap_servers = ['localhost:9092']
topic = 'your_kafka_topic'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

# Configure Cassandra
contact_points = ['localhost']
auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(contact_points=contact_points, auth_provider=auth_provider)
session = cluster.connect()


# Envoie du dataFrame vers Kafka
serialized_df = data.to_json().encode('utf-8')  # encode the JSON string to bytes
producer.send(topic, value=serialized_df)

# Le dataFrame est ensuite stocké dans la base de données Cassandra
session.execute("CREATE KEYSPACE IF NOT EXISTS your_keyspace WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}")
session.execute("USE your_keyspace")
session.execute("CREATE TABLE IF NOT EXISTS RSS (title text, pubdate text, link text, description text, PRIMARY KEY(pubDate)) WITH CLUSTERING ORDER BY (pubDate DESC)")

for _, row in data.iterrows():
    session.execute(f"INSERT INTO RSS (title, pubdate, link, description) VALUES ('{row['title']}', '{row['pubDate']}', '{row['link']}', '{row['description']}')")


In [ ]:
rows = session.execute("SELECT * FROM RSS")
for row in rows:
    print(row.link)

In [ ]:
#Ferme la connection
def close_connection():
    producer.flush()
    producer.close()
    session.shutdown()
    cluster.shutdown()
    producer.flush()
    producer.close()
    session.shutdown()
    cluster.shutdown()

In [19]:
#Suppression des données dans la basse de données
def delete_data():
    session.execute("DESCRIBE KEYSPACES;")
    session.execute("USE your_keyspace;")
    session.execute("DESCRIBE TABLES;")
    session.execute("TRUNCATE RSS")

In [36]:
delete_data()